In [124]:
!pip install --user --upgrade beautifulsoup4 lxml html5lib==1.0b8

Requirement already up-to-date: beautifulsoup4 in ./.local/lib/python3.5/site-packages
Requirement already up-to-date: lxml in ./.local/lib/python3.5/site-packages
Requirement already up-to-date: html5lib==1.0b8 in ./.local/lib/python3.5/site-packages
Requirement already up-to-date: six in ./.local/lib/python3.5/site-packages (from html5lib==1.0b8)


In [163]:
import requests
from bs4 import BeautifulSoup

In [162]:
def funtion(url,cookies):
    resp = requests.get(
        url, 
        cookies=cookies
    )

    soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

    gossip_list = soup.find('div', {'class': 'r-list-container action-bar-margin bbs-screen'})

    a_list = gossip_list.find_all('a')
    #print(type(a_list))

    bbs_url_list = [link.get('href') for link in a_list]

    bbs_text_list = [link.text for link in a_list]

    author_list = gossip_list.find_all('div', {'class': 'author'})
    author_list = [link.text for link in author_list]
    
    re_dic = {
        'bbs_url_list': bbs_url_list,
        'bbs_text_list': bbs_text_list,
        'author_list': author_list
    }
    
    return re_dic


In [161]:
# up_url_list = ['1','2','3','4','5']
up_url_list = []

url = "https://www.ptt.cc/bbs/Gossiping/index.html"

for i in range(0, 5):
    resp = requests.get(
        url, 
        cookies={'over18': '1'}
    )
    soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

    page_list = soup.find('div', {'class': 'btn-group btn-group-paging'})
    up_list = page_list.find_all('a')
#     up_url_list[i] = 'https://www.ptt.cc' + up_list[1].get('href')
    up_url_list.append('https://www.ptt.cc' + up_list[1].get('href'))
    url = up_url_list[i]
    
page_info = []

for i in range(0, 5):
    print(up_url_list[i])
    page_info.append(funtion(up_url_list[i],{'over18': '1'}))

for i in range(0, 5):
    print(len(page_info[i]['bbs_url_list']))
    print(len(page_info[i]['bbs_text_list']))
    print(len(page_info[i]['author_list']))
    print(' ')
    
    
    

https://www.ptt.cc/bbs/Gossiping/index29647.html
https://www.ptt.cc/bbs/Gossiping/index29646.html
https://www.ptt.cc/bbs/Gossiping/index29645.html
https://www.ptt.cc/bbs/Gossiping/index29644.html
https://www.ptt.cc/bbs/Gossiping/index29643.html
23
23
24
 
20
20
20
 
20
20
20
 
19
19
20
 
19
19
20
 


In [216]:
class PttCrawler:
    def __init__(self, board, write= False ):
        self.ptt_url = 'https://www.ptt.cc'
        self.board = board
        self.now_page = board
        self.session = requests.Session()
        self.session.cookies.update({
            'over18':'1'
        })
        self.write=write
        
        
    def page_info(self,url):
        resp = self.session.get(
            url
        )

        soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

        gossip_list = soup.find('div', {'class': 'r-list-container action-bar-margin bbs-screen'})

        a_list = gossip_list.find_all('div',{'r-ent'})
        
        re_dic = []
        
        for link in a_list:
            bbs_a = link.find('a')
            
            author = link.find('div', {'class': 'author'})
            date = link.find('div', {'class': 'date'})
            
            bbs_url = 'https://www.ptt.cc' + bbs_a.get('href')
    
            resp = self.session.get(
                bbs_url
            )
            soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')
            
            bbs_content = soup.find('div',{'id': 'main-content'})
            
            content = ''
            s = list(bbs_content.strings)
            for i in range(8,len(s)-1):
                content = content + s[i]
            
            re_dic.append({
                'bbs_url': bbs_url,
                'bbs_text': bbs_a.text,
                'author': author.text,
                'date': date.text,
                'content': content
            })
        
        return re_dic
    
    def previous_page_url(self,url):
        resp = self.session.get(
            url
        )
        soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

        page_list = soup.find('div', {'class': 'btn-group btn-group-paging'})
        up_list = page_list.find_all('a')
        if not up_list[1].get('href'):
            re_str = 'this is first page!'
        else:
            re_str = 'https://www.ptt.cc' + up_list[1].get('href')
            self.now_page = re_str
        return self.now_page
    
    def next_page_url(self,url):
        resp = self.session.get(
            url
        )
        soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

        page_list = soup.find('div', {'class': 'btn-group btn-group-paging'})
        up_list = page_list.find_all('a')
        if not up_list[2].get('href'):
            re_str = 'this is last page!'
        else:
            re_str = 'https://www.ptt.cc' + up_list[2].get('href')
            self.now_page = re_str
        return self.now_page
    
    def to_page(self,page):
        re_str = self.board
        for i in range(0, page):
            re_str = self.previous_page(re_str)
        self.now_page = re_str
        return self.now_page
    
    @property
    def show_now_page(self):
        return self.now_page
    
    @property
    def next_page(self):
        self.next_page_url(self.now_page)
        return self.now_page
    
    @property
    def previous_page(self):
        self.previous_page_url(self.now_page)
        return self.now_page
    
    @property
    def return_to_board(self):
        self.now_page = self.board
        return self.now_page


In [217]:
test = PttCrawler('https://www.ptt.cc/bbs/Gossiping/index.html')
print(test.previous_page_url('https://www.ptt.cc/bbs/Gossiping/index.html'))
print(test.previous_page)
print(test.next_page)
print(test.now_page)
print(test.return_to_board)

re_dic = test.page_info(test.now_page)
print(len(re_dic))
print(re_dic[0]['bbs_text'])
print(re_dic[0]['bbs_url'])
print(re_dic[0]['author'])
print(re_dic[0]['content'])
print(re_dic[0]['date'])

https://www.ptt.cc/bbs/Gossiping/index29652.html
https://www.ptt.cc/bbs/Gossiping/index29651.html
https://www.ptt.cc/bbs/Gossiping/index29652.html
https://www.ptt.cc/bbs/Gossiping/index29652.html
https://www.ptt.cc/bbs/Gossiping/index.html
13
[問卦] 延續剛剛酒駕事件
https://www.ptt.cc/bbs/Gossiping/M.1513851901.A.CC7.html
chamge7788

※ 八卦板務請到 GossipPicket 檢舉板詢問
※ a.張貼問卦請注意，充實文章內容、是否有專板，本板並非萬能問板。
※ b.一天只能張貼 "兩則" 問卦，自刪及被刪也算兩篇之內，
※   超貼者將被水桶，請注意!
※ c.本看板嚴格禁止政治問卦，發文問卦前請先仔細閱讀相關板規。
※ d.未滿30繁體中文字視同鬧板，請注意！
※ (↑看完提醒請刪除ctrl + y)
各位鄉民大家好 我剛剛看了很多人噓文 首先酒駕不對 但是當天回收很多非常多

我爸只是幫忙載而已 當天我可以說

我爸並沒有一喝酒 就開車

而是 過了11小時才去幫忙

當天是週六 車被寄放在拖到場所

過年我們家都不好過 做業務的 只因為幫忙載回收 沒有一喝酒就開車 因為是過年

所以無法當天領車 而且 我們家在那時候

只有爸爸有駕照 現在沒有了 由我哥來開車 我哥駕照考了 11次才過

很難 尤其是路考 連我方向感很好的人

都68分沒過 回到正題 而且我是左撇子喔

左撇子沒有人路考一次過的

我可以說 回收過多導致我們家 經濟受影響 但是酒駕就是不對 但是以經過了11小時才開
車 而且是幫忙載回收喔 卻要掉扣

駕照 因為是業務員 不能開車了 害我哥只能去上班當作業原 無法考公務員 因為當時家
中沒有經濟支柱

總結 如果當下喝酒開車 不該原諒是垃圾

撞傷人 判死或鞭刑

如果有過10小時以上 但是酒測值0.24以下 一公里內 且做善事著 可只罰錢 不用掉扣駕
照 這對一個家庭影響很大 而且駕照很難考 尤其是路考

有沒有惡酒駕根善酒駕的卦


In [207]:
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

resp = requests.get(
    url, 
    cookies={'over18': '1'}
)
soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')

gossip_list = soup.find('div', {'class': 'r-list-container action-bar-margin bbs-screen'})

r_list = gossip_list.find_all('div',{'class': 'r-ent'})

re_dic = []

for link in r_list:
    bbs_a = link.find('a')
    author = link.find('div', {'class': 'author'})
    
    bbs_url = 'https://www.ptt.cc' + bbs_a.get('href')
    
    resp = requests.get(
        bbs_url,
        cookies={'over18': '1'}
    )
    soup = BeautifulSoup(resp.text.encode('utf-8'), 'lxml')
    bbs_content = soup.find('div',{'id': 'main-content'})
    
    content = ''
    s = list(bbs_content.strings)
    for i in range(8,len(s)-1):
        content = content + s[i]
    
    re_dic.append({
        'bbs_url': bbs_a.get('href'),
        'bbs_text': bbs_a.text,
        'author': author.text,
        'content': content
    })
    
print(len(re_dic))
print(re_dic[0]['bbs_text'])
print(re_dic[0]['bbs_url'])
print(re_dic[0]['author'])
print(re_dic[0]['content'])

11
[問卦] 忠孝東路為什麼要走九遍
/bbs/Gossiping/M.1513851291.A.FD7.html
alan4ni

大家好

我是從我家走到巷口就快累死的艾倫

聽說啊動力火車有一首歌叫做忠孝東路走九遍

但忠孝東路真的是超級長的起點從監察院到南港都還是忠孝東路

一趟走完起碼要3 4個小時吧

為什麼還要走九遍勒 幹嘛不挑南京東路 或 民權東路勒

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 117.19.73.223
※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1513851291.A.FD7.html
